# Data preprocessing

The dataset is composed of three parts:
1. descriptors of **MOFs** (under `data/ML_data`)
2. descriptors of **adsorbants** (mannually added below)
3. **adsorption uptakes** of **(MOF, adsorbate) pairs** (under `data/flexibility_data/y_data/adsorption_data`), containing two values:
    1. values from rigid model
    2. mean values from flexible model

## read MOF data

In [80]:
import pandas as pd
import numpy as np
import os

# read the 36-descriptor data
df36Descriptor = pd.read_excel('data/ML_data/descriptor_used.xlsx',header=4,index_col=1)
df36Descriptor.head()

,Data point ID,MOF,Molecule,ρ(g.cm-3),PLD (Å),vf,vp (cm3.g-1),V (A3),nAT-H,nNM,...,Pc (bar),omega,Pr,q (mol.kg-1) / GCMC simulations (Tang et al. ),q est (mol.kg-1) / Tang et al.,q pred (mol.kg-1) / Eq. (S1),Status / Eq. (S1),AD,ADpred / Eq. (S2),Status / Eq. (S2)
Isotherm ID,,,,,,,,,,,,,,,,,,,,,
1,1,ABUWOJ,2-Pentanone,1.15833,4.03039,0.545974,0.532253,4354.2656,168,1.238095,...,36.928860,0.349050,0.014028,2.419113,2.048210,2.840610,training,1,1,training
1,2,ABUWOJ,2-Pentanone,1.15833,4.03039,0.545974,0.532253,4354.2656,168,1.238095,...,36.928860,0.349050,0.001403,2.301501,2.043625,2.800204,validation,0,1,test
1,3,ABUWOJ,2-Pentanone,1.15833,4.03039,0.545974,0.532253,4354.2656,168,1.238095,...,36.928860,0.349050,0.000140,2.387922,1.998884,2.512170,validation,1,1,training
1,4,ABUWOJ,2-Pentanone,1.15833,4.03039,0.545974,0.532253,4354.2656,168,1.238095,...,36.928860,0.349050,0.000014,2.194952,1.639868,1.465350,training,0,0,validation
2,5,ABUWOJ,Acetaldehyde,1.15833,4.03039,0.545974,0.532253,4354.2656,168,1.238095,...,53.982845,0.288583,0.215029,4.801622,5.365729,5.573014,test,1,1,training


In [112]:
# obtain the descriptor list
columns = [df36Descriptor.columns[1]] + df36Descriptor.columns[3: -7].tolist()
print(columns)

['MOF', 'ρ(g.cm-3)', 'PLD (Å)', 'vf', 'vp (cm3.g-1)', 'V (A3)', 'nAT-H', 'nNM', 'nM', 'nTB', 'nSB', 'nMB', 'nRB', 'nR6', 'nTrM', 'nDB', 'nAcyclB', 'nR8', 'nAlkylC', 'nVinylC', 'nEnamineAN', 'nOHEPh', 'nR5', 'nR4', 'MType', 'MaxMVal', 'n-O-', 'F01[H-C]', 'F01[C-N]', 'F01[C-O]', 'F02[H-C]', 'F02[C-N]', 'F02[C-O]', 'Tc (K)', 'Pc (bar)', 'omega', 'Pr']


In [113]:
# clean up columns
newColumns = []
for ci in columns:
    if ' ' in ci:
        newColumns.append(ci.split(' ',1)[0])
    elif '(' in ci:
        newColumns.append(ci.split('(',1)[0])
    else:
        newColumns.append(ci)
print(newColumns)

['MOF', 'ρ', 'PLD', 'vf', 'vp', 'V', 'nAT-H', 'nNM', 'nM', 'nTB', 'nSB', 'nMB', 'nRB', 'nR6', 'nTrM', 'nDB', 'nAcyclB', 'nR8', 'nAlkylC', 'nVinylC', 'nEnamineAN', 'nOHEPh', 'nR5', 'nR4', 'MType', 'MaxMVal', 'n-O-', 'F01[H-C]', 'F01[C-N]', 'F01[C-O]', 'F02[H-C]', 'F02[C-N]', 'F02[C-O]', 'Tc', 'Pc', 'omega', 'Pr']


In [68]:
# read ML data
dfMLOrigin = pd.read_excel('data/ML_data/descriptor_4717MOF.xlsx')

The dataset contains 4717 MOFs with 1024 features.

In [114]:
MLColumns = dfMLOrigin.columns.tolist()

newMLColumns = {}

for ci in MLColumns:
    if ' ' in ci:
        newMLColumns[ci] = ci.split(' ',1)[0]
    elif '(' in ci:
        newMLColumns[ci] = ci.split('(',1)[0]
    else:
        newMLColumns[ci] = ci
dfMLOriginShortNames = dfMLOrigin.rename(columns=newMLColumns)

dfMLOriginShortNames.head()

,ID,MOF,Periodic,ρ,PLD,LCD,VSA,GSA,vf,vp,...,F01[Ne-Rh],F01[Ne-Pd],F01[Ne-Ag],F01[Ne-Cd],F01[Ne-In],F01[Ne-Sn],F01[Ne-Sb],F01[Ne-Te],F01[Ne-I],F01[Ne-Xe]
0,1,ABAVIJ,H8C12N2O4Co,1.52493,2.49720,4.45543,0.000,0.000,0.234030,0.255564,...,0,0,0,0,0,0,0,0,0,0
1,2,ABAVOP,H8C12N2O4Co,1.56594,2.44162,3.53642,0.000,0.000,0.172707,0.239596,...,0,0,0,0,0,0,0,0,0,0
2,3,ABEMIF,H8C24O16ClCu4,1.15199,6.80253,11.25251,1185.300,1028.920,0.686068,0.556041,...,0,0,0,0,0,0,0,0,0,0
3,4,ABEXEM,H2C7N2O6La,1.76082,4.14331,4.82084,646.239,367.010,0.349898,0.277374,...,0,0,0,0,0,0,0,0,0,0
4,5,ABEXIQ,H2C7N2O6Ce,1.78760,4.08045,4.80529,591.710,331.008,0.344895,0.270205,...,0,0,0,0,0,0,0,0,0,0


In [115]:
# only use 36-descriptor columns
shared_descriptor = [col for col in dfMLOriginShortNames.columns if col in newColumns]
dfMLReduced = dfMLOriginShortNames[shared_descriptor]
dfMLReduced.head()

,MOF,ρ,PLD,vf,vp,V,nAT-H,nNM,nM,nTrM,...,nR6,nR8,nAlkylC,nVinylC,nEnamineAN,nOHEPh,n-O-,F01[H-C],F01[C-N],F01[C-O]
0,ABAVIJ,1.52493,2.49720,0.234030,0.255564,1320.3834,76,1.368421,0.052632,0.052632,...,0.105263,0.052632,0.000000,0.631579,0.210526,0.210526,0.210526,0.421053,0.210526,0.210526
1,ABAVOP,1.56594,2.44162,0.172707,0.239596,1285.8069,76,1.368421,0.052632,0.052632,...,0.105263,0.052632,0.000000,0.631579,0.210526,0.210526,0.210526,0.421053,0.210526,0.210526
2,ABEMIF,1.15199,6.80253,0.686068,0.556041,3640.8920,135,1.088889,0.088889,0.088889,...,0.237037,0.088889,0.533333,0.000000,0.000000,0.000000,0.355556,0.177778,0.000000,0.355556
3,ABEXEM,1.76082,4.14331,0.349898,0.277374,2633.0447,128,1.062500,0.062500,0.062500,...,0.062500,0.031250,0.093750,0.312500,0.125000,0.250000,0.343750,0.125000,0.250000,0.375000
4,ABEXIQ,1.78760,4.08045,0.344895,0.270205,2602.5916,128,1.062500,0.062500,0.000000,...,0.062500,0.031250,0.093750,0.312500,0.125000,0.250000,0.343750,0.125000,0.250000,0.375000


The reduced dataset contains 4717 MOFs with 29 features (excluding the first column).

## read adsorption update data

In [116]:
# the MOFs in "dfMLReduced" and adsorption data sets are different, so it is necessary to match the MOFs in two datasets
def datasetMatch(MOFName):
    dfML= dfMLReduced[dfMLReduced['MOF'].isin(MOFName)]
    matchedMOFIndex=np.isin(MOFName, dfML['MOF'].values)
    return matchedMOFIndex, dfML

# read flexibility data
flexibilityList=os.listdir('data/flexibility_data/y_data/adsorption_data') # obtain list of csv files for 9 adsorption uptakes
flexivilityData=[]
adsorbantNameList = []

for i, name in enumerate(flexibilityList):
    # read csv files for certain adsorption uptakes
    df = pd.read_csv('data/flexibility_data/y_data/adsorption_data/' + name)
    
    # obtain the rigid value
    rigidValue = np.array(df[df.columns[1]], dtype = float)
    
    # obtain the flexible mean value
    flexValue = np.mean(np.array(df[df.columns[2:]],dtype=float),axis=1)
    
    # obtain the adsorbate label
    label = np.array([name.split("_")[1] for x in range(0,len(flexValue))],dtype=str)
    adsorbantNameList.append(name.split("_")[1])
    
    # stack the rigid value, flexible mean value and the adsorbate label
    singleSet = np.column_stack([rigidValue,flexValue,label])

    if i == 0:
        # obtain the name list of MOFs
        MOFNaemTemp = np.array(df[df.columns[0]], dtype = str)
        MOFName = [x.split("_")[0] for x in MOFNaemTemp]
        
        # search the MOF name in "dfMLReduced", generating dfML
        matchedMOFIndex, dfML = datasetMatch(MOFName)
        print("The number of MOFs shared by two datasets are: {:d}.\n".format(dfML.shape[0]))
        
        # generating flexibilityData as "y"
        flexibilityData = singleSet[matchedMOFIndex,:].copy()
    else:
        # concatenate "y"
        flexibilityData = np.concatenate([flexibilityData.copy(),singleSet[matchedMOFIndex,:].copy()])

flexibilityData

The number of MOFs shared by two datasets are: 98.



array([['3.804813548', '3.921763156704', 'propane'],
       ['4.613223308', '3.951063714919', 'propane'],
       ['1.554248904', '3.010842269951917', 'propane'],
       ...,
       ['18.384843', '18.130595944004163', 'ethene'],
       ['6.886174479', '6.452062477688332', 'ethene'],
       ['6.881538807999999', '6.442196383918749', 'ethene']], dtype='<U32')

In [117]:
print(flexibilityData.shape)

(882, 3)


flexibilityData contains the adsorption update data for (MOF, adsorbate) pairs. There are 98 MOFs and 9 adsorbants, so there are 882 data points in total.
- 1st column: rigid data
- 2nd column: flexible mean data
- 3rd column: adsorbate label

The order of the flexibilityData is:

| MOF | adsorbant |
|------|------------|
| MOF1 | adsorbant1 |
| MOF2 | adsorbant1 |
| MOF3 | adsorbant1 |
| ...  | ...        |
| MOF98 | adsorbant1 |
| MOF1 | adsorbant2 |
| MOF2 | adsorbant2 |
| MOF3 | adsorbant2 |
| ...  | ...        |
| MOF98 | adsorbant2 |
| MOF1 | adsorbant3 |
| MOF2 | adsorbant3 |
| MOF3 | adsorbant3 |
| ...  | ...        |

## manually add adsorbant data

In [145]:
# manually add adsorbate descriptors

# Mw/gr.mol-1, Tc/K, Pc/bar, ω, Tb/K, Tf/K

adsorbateData=np.array([
    ['xenon',131.293,289.7,58.4,0.008,164.87,161.2], 
    ['butane',58.1,449.8,39.5,0.3,280.1,146.7], 
    ['propene',42.1,436.9,51.7,0.2,254.8,150.6], 
    ['ethane',30.1,381.8,50.3,0.2,184.0,126.2], 
    ['propane',44.1,416.5,44.6,0.2,230.1,136.5], 
    ['CO2',44.0,295.9,71.8,0.2,317.4,204.9], 
    ['ethene',28.054,282.5,51.2,0.089,169.3,228], 
    ['methane',16.04,190.4,46.0,0.011,111.5,91],
    ['krypton',83.798,209.4,55.0,0.005,119.6,115.6]])

adsorbateData.shape
adDf = pd.DataFrame(data=adsorbateData, columns=["adsorbant", "Mw/gr.mol-1", "Tc/K", "Pc/bar", "ω", "Tb/K", "Tf/K"])

# sort the dataframe based on adsorbantNameList
sorterIndex = dict(zip(adsorbantNameList,range(len(adsorbantNameList))))
adDf['an_Rank'] = adDf['adsorbant'].map(sorterIndex)
adDf.sort_values(['an_Rank'],ascending = [True], inplace = True)
adDf.drop('an_Rank', 1, inplace = True)
adDfFloat = adDf.iloc[:, 1:].astype(np.float)
adDfFloat["adsorbant"] = adDf["adsorbant"]
adDfFloat

,Mw/gr.mol-1,Tc/K,Pc/bar,ω,Tb/K,Tf/K,adsorbant
4,44.100,416.5,44.6,0.200,230.10,136.5,propane
1,58.100,449.8,39.5,0.300,280.10,146.7,butane
5,44.000,295.9,71.8,0.200,317.40,204.9,CO2
8,83.798,209.4,55.0,0.005,119.60,115.6,krypton
3,30.100,381.8,50.3,0.200,184.00,126.2,ethane
2,42.100,436.9,51.7,0.200,254.80,150.6,propene
0,131.293,289.7,58.4,0.008,164.87,161.2,xenon
7,16.040,190.4,46.0,0.011,111.50,91.0,methane
6,28.054,282.5,51.2,0.089,169.30,228.0,ethene


In [147]:
print(adDfFloat.shape)

(9, 7)


There are 6 descriptors (excluding name label) for each adsorbant.

## combine MOF and adsorbant descriptors
The combined dataset should have $29+6=35$ descriptors:

In [148]:
# replicate dfML for 9 adsorbants
dfMLReplicate = pd.concat([dfML]*9)

# replicate adDf for 98 MOFs
adDfReplicate = pd.DataFrame(np.repeat(adDfFloat.values,98,axis=0))
adDfReplicate.columns = adDfFloat.columns

# concatenate two datasets
dfMLReplicate.reset_index(drop=True, inplace=True)
adDfReplicate.reset_index(drop=True, inplace=True)
XAllDescriptor = pd.concat([dfMLReplicate, adDfReplicate],axis=1)
print(XAllDescriptor.shape)
XAllDescriptor.head()

(882, 37)


,MOF,ρ,PLD,vf,vp,V,nAT-H,nNM,nM,nTrM,...,F01[H-C],F01[C-N],F01[C-O],Mw/gr.mol-1,Tc/K,Pc/bar,ω,Tb/K,Tf/K,adsorbant
0,ABUWOJ,1.158330,4.03039,0.545974,0.532253,4518.15800,168,1.238095,0.095238,0.095238,...,0.285714,0.000000,0.285714,44.1,416.5,44.6,0.2,230.1,136.5,propane
1,ACOLIP,1.049490,3.57647,0.454051,0.521040,1527.85830,64,1.562500,0.031250,0.031250,...,0.531250,0.343750,0.125000,44.1,416.5,44.6,0.2,230.1,136.5,propane
2,AGARUW,1.771950,6.25183,0.450504,0.291150,2421.46580,112,1.071428,0.071429,0.071429,...,0.071429,0.142857,0.392857,44.1,416.5,44.6,0.2,230.1,136.5,propane
3,AHOKIR01,1.927010,3.46842,0.460404,0.251926,293.96747,14,1.428572,0.142857,0.142857,...,0.571429,0.000000,0.000000,44.1,416.5,44.6,0.2,230.1,136.5,propane
4,AMILUE,0.982365,11.07263,0.566397,0.622654,4728.39160,176,1.500000,0.045455,0.045455,...,0.545455,0.272727,0.181818,44.1,416.5,44.6,0.2,230.1,136.5,propane


## generate X and y
The rigid uptake data can be added into X:

In [158]:
X = np.concatenate((XAllDescriptor.iloc[:, 1:-1], flexibilityData[:, 0].reshape(-1, 1)),axis=1).astype(np.float)
print(X.shape)

(882, 36)


The flexible mean data is chosen as y:

In [159]:
y = flexibilityData[:, 1]
print(y.shape)

(882,)


# Validation set split

In [187]:
# ----------------------------------------------------------------------------------------
# ---------------------------- don't touch the validation set ----------------------------
np.random.seed(0)
from sklearn.model_selection import train_test_split
X_train_test, X_validation, y_train_test, y_validation = train_test_split(X, y, test_size=0.25)
# ---------------------------- don't touch the validation set ----------------------------
# ----------------------------------------------------------------------------------------

# Regression